In [1]:
import os
import re
import gensim
import string
import nltk
import pickle
import numpy as np
import pandas as pd
from numpy import percentile
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import preprocessing_module

In [3]:
import warnings
warnings.filterwarnings("ignore")                     
import pandas as pd
import os

# Step 1: Load the corpus
content = []
file_names = []
path = '/mnt/c/Users/alamm9/Desktop/text_file_train_test/train_set'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as file:
        content.append(file.read())
        file_names.append(filename)

# Step 2: Create a dataframe from the corpus
df_train = pd.DataFrame({'Filename': file_names, 'Content': content})

In [4]:
df_train = preprocessing_module.preprocess_text(df_train, "Content")

In [5]:
from sklearn.model_selection import train_test_split

# Splitting the dataset
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=40)

In [6]:
def w2v(df_train,df):
    
    model = gensim.models.Word2Vec(df_train["Content"], vector_size=100, window=5, min_count=1, workers=4)
#     with open("model.pkl", "wb") as f:
#         pickle.dump(model, f)
    
    # Obtain document embeddings for the training set
    train_embeddings = [np.mean([model.wv[token] for token in doc_tokens if token in model.wv], axis=0) for doc_tokens in df_train["Content"]]
#     with open("train_embeddings(w2v).pkl", "wb") as f:
#         pickle.dump(train_embeddings, f)
#         print(train_embeddings[0])
#         print(len(train_embeddings))
#         print(train_embeddings[0].shape)
    
    
    # Transform the test set embeddings using the learned transformation from the training set
    test_embeddings = [np.mean([model.wv[token] for token in doc_tokens if token in model.wv], axis=0) for doc_tokens in df["Content"]]

    # Calculate cosine similarity matrix
    similarity_matrix = cosine_similarity(train_embeddings, test_embeddings) #trian 80% and validation 20%
    print("similarity_matrix",similarity_matrix.shape)

#     print(similarity_matrix[0])



    # Calculate mean similarity for each row
    mean_similarity = np.mean(similarity_matrix, axis=0)
#     print(mean_similarity)
#     print(mean_similarity.shape)
    threshold = np.percentile(mean_similarity, [10])
    print("threshold(w2v):", threshold)
#     with open("threshold(w2v).pkl", "wb") as f:
#         pickle.dump(threshold, f)


#************************************************************************************************************************

def tfidf(df_train,df):
    
    # Initialize the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    
    # Fitting vectorizer in the training dataset
    train = vectorizer.fit_transform(df_train["Content"])

    # with open("train(tfidf).pkl", "wb") as f:
    #     pickle.dump(train, f)
    #    print("train(tfidf):", train)

    
    # with open("vectorizer.pkl", "wb") as f:
    #     pickle.dump(vectorizer, f)
    #    print("vectorizer(tfidf):", vectorizer)


    # Apply the same vectorizer on the test documents
    test = vectorizer.transform(df["Content"])

    similarity_matrix1 = cosine_similarity(train, test) #trian 80% and validation 20%
    print("similarity_matrix",similarity_matrix1.shape)

    # Calculate mean similarity for each row
    mean_similarity1 = np.mean(similarity_matrix1, axis=0)
    # print(mean_similarity1)
    # print(mean_similarity1.shape)
    threshold_tfidf = np.percentile(mean_similarity1, 10)
    print("threshold(tfidf):", threshold_tfidf)
    # with open("threshold(tfidf).pkl", "wb") as f:
    #      pickle.dump(threshold_tfidf, f)

In [9]:
#tfidf(df_train,df_validation)

In [12]:
#w2v(df_train,df_validation)